<div style="text-align:center; color:  #13476a ; font-size:20px">

<h1> Api twitter  </h1> 

Alejandro Fandiño B
</div>

- Extraer Tuits

- Hacer una visualización sobre esos tuits


In [3]:
import pandas as pd
import tweepy
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
 
import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

## Punto 1 - extraer tweets

In [4]:
keys = pd.read_csv('keys_demo.csv', sep=';', header=None)
keys = dict(zip(keys[0],keys[1]))

In [5]:
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_key_secret'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [6]:
tweets = tweepy.Cursor(api.search,
                       q=['claudia lopez -RT'],
                       lang='es',
                       geocode=f'4.673495,-74.1316,45mi',
                       tweet_mode='extended',
                       include_rts=False).items(500)

In [7]:
t = []
 
for tweet in tweets:
    temp = []
    temp.append(tweet.full_text)
    temp.append(tweet.created_at)
    temp.append("Bogota")
    temp.append(tweet.user.location)
    temp.append(tweet.place)
    temp.append(tweet.coordinates)
    temp.append(tweet.id)
    temp.append(f"https://twitter.com/i/web/status/{tweet.id}")

    t.append(temp)

In [8]:
data = pd.DataFrame(t)
data.columns = ['tweet','date', 'city', 'user_location', 'place', 'coordinates', 'id', 'link']
data.head()

,tweet,date,city,user_location,place,coordinates,id,link
0,"#Seguimiento 🔎| La alcaldesa de Bogotá, Claudi...",2020-11-12 15:00:01,Bogota,Bogotá,None,None,1326902577020063744,https://twitter.com/i/web/status/1326902577020...
1,@walter0244 @CesarPachonAgro @lilicatadiaz @Cl...,2020-11-12 14:51:35,Bogota,"Bogotá, Colombia",None,None,1326900452437479435,https://twitter.com/i/web/status/1326900452437...
2,Claudia López insiste en realizar reforma a la...,2020-11-12 14:35:03,Bogota,"Bogotá, Colombia",None,None,1326896293084590083,https://twitter.com/i/web/status/1326896293084...
3,Deber de policías era salvarles la vida: @Clau...,2020-11-12 14:34:27,Bogota,"Bogotá, Colombia",None,None,1326896143129776129,https://twitter.com/i/web/status/1326896143129...
4,"Pues el discurso del odio no solo es de Petro,...",2020-11-12 14:26:01,Bogota,"Armenia, Colombia",None,None,1326894018362552321,https://twitter.com/i/web/status/1326894018362...


In [9]:
data.shape

(500, 8)

In [10]:
data.to_csv('datos.csv',index=False)

In [12]:
data = pd.read_csv('datos.csv', sep=',')
data.head()

,tweet,date,city,user_location,place,coordinates,id,link
0,"#Seguimiento 🔎| La alcaldesa de Bogotá, Claudi...",2020-11-12 15:00:01,Bogota,Bogotá,NaN,NaN,1326902577020063744,https://twitter.com/i/web/status/1326902577020...
1,@walter0244 @CesarPachonAgro @lilicatadiaz @Cl...,2020-11-12 14:51:35,Bogota,"Bogotá, Colombia",NaN,NaN,1326900452437479435,https://twitter.com/i/web/status/1326900452437...
2,Claudia López insiste en realizar reforma a la...,2020-11-12 14:35:03,Bogota,"Bogotá, Colombia",NaN,NaN,1326896293084590083,https://twitter.com/i/web/status/1326896293084...
3,Deber de policías era salvarles la vida: @Clau...,2020-11-12 14:34:27,Bogota,"Bogotá, Colombia",NaN,NaN,1326896143129776129,https://twitter.com/i/web/status/1326896143129...
4,"Pues el discurso del odio no solo es de Petro,...",2020-11-12 14:26:01,Bogota,"Armenia, Colombia",NaN,NaN,1326894018362552321,https://twitter.com/i/web/status/1326894018362...


In [13]:
import re
 
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'
 
def text_clean(row):
    text = row['tweet']

    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")

    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention,"")

    return text, links, mentions

In [14]:
data[['clean', 'links', 'mentions']] = data.apply(text_clean, axis=1, result_type='expand')

data.head()

,tweet,date,city,user_location,place,coordinates,id,link,clean,links,mentions
0,"#Seguimiento 🔎| La alcaldesa de Bogotá, Claudi...",2020-11-12 15:00:01,Bogota,Bogotá,NaN,NaN,1326902577020063744,https://twitter.com/i/web/status/1326902577020...,"#Seguimiento 🔎| La alcaldesa de Bogotá, Claudi...",[https://t.co/7pltEe73Sj],[]
1,@walter0244 @CesarPachonAgro @lilicatadiaz @Cl...,2020-11-12 14:51:35,Bogota,"Bogotá, Colombia",NaN,NaN,1326900452437479435,https://twitter.com/i/web/status/1326900452437...,"No la defiendo, pero Claudia Lopez es la ...",[],"[@walter0244, @CesarPachonAgro, @lilicatadiaz,..."
2,Claudia López insiste en realizar reforma a la...,2020-11-12 14:35:03,Bogota,"Bogotá, Colombia",NaN,NaN,1326896293084590083,https://twitter.com/i/web/status/1326896293084...,Claudia López insiste en realizar reforma a la...,"[https://t.co/YW2FskBtb9, https://t.co/815gLl7...",[]
3,Deber de policías era salvarles la vida: @Clau...,2020-11-12 14:34:27,Bogota,"Bogotá, Colombia",NaN,NaN,1326896143129776129,https://twitter.com/i/web/status/1326896143129...,Deber de policías era salvarles la vida: por ...,[https://t.co/i6CJ4GOJ19],[@ClaudiaLopez]
4,"Pues el discurso del odio no solo es de Petro,...",2020-11-12 14:26:01,Bogota,"Armenia, Colombia",NaN,NaN,1326894018362552321,https://twitter.com/i/web/status/1326894018362...,"Pues el discurso del odio no solo es de Petro,...",[https://t.co/8GjPXJZeD4],[]


## Punto 2 - visualización

### Hashtags

In [15]:
import re
def find_hash(text):
    line=re.findall(r'(?<=#)\w+',text)
    return " ".join(line)

data['hashtags']=data['tweet'].apply(lambda x:find_hash(x))

In [16]:
from collections import Counter
hastags=list(data[(data['hashtags'].notnull())&(data['hashtags']!="")]['hashtags'])
hastags = [each_string.lower() for each_string in hastags]
hash_df=dict(Counter(hastags))
top_hash_df=pd.DataFrame(list(hash_df.items()),columns = ['word','count']).sort_values('count',ascending=False)
top_hash_df.head(10)

,word,count
10,noticiascapital,7
44,infobogotá,4
4,revocatoriaclaudialopez,4
33,economíanaranja envivo,4
39,voces,3
24,bogotá,3
0,seguimiento,2
18,diasincarne,2
43,elecciones2020,2
37,entrevistas,2


In [17]:
fig = go.Figure(go.Bar(
    x=top_hash_df['word'],y=top_hash_df['count'],
    marker={'color': top_hash_df['count'], 
    'colorscale': 'blues'},  
    text=top_hash_df['count'],
    textposition = "outside",

))

fig.update_layout(title_text='Top Hastags',xaxis_title="Hashtags",
                  yaxis_title="Número de Tags ",template="plotly_white",height=700,title_x=0.5)
fig.show()

## Serie de tiempo

In [23]:
data['date'] =  pd.to_datetime(data['date'])
data['fecha'] = data['date'].dt.date
fecha_df = pd.DataFrame(data.fecha.value_counts()).reset_index()
fecha_df.columns = ['fecha', 'cnt']
fecha_df = fecha_df.sort_values('fecha')
fecha_df

,fecha,cnt
6,2020-11-06,17
0,2020-11-07,115
2,2020-11-08,76
4,2020-11-09,67
1,2020-11-10,112
3,2020-11-11,72
5,2020-11-12,41


In [24]:
import plotly.graph_objs as go
fig=go.Figure(go.Scatter(x=fecha_df['fecha'],
                                y=fecha_df['cnt'],
                               mode='markers+lines',
                               name="Submissions",
                               marker_color='dodgerblue'))

fig.update_layout(
    title_text='Tweets por día : ({} - {}) '.format(fecha_df['fecha'].sort_values()[0].strftime("%d/%m/%Y"),
                                                       fecha_df['fecha'].sort_values().iloc[-1].strftime("%d/%m/%Y")),template="plotly_white",
    title_x=0.5)

fig.show()

Este taller se realizo en grupo con Daileny Dorado